## Import Libraries
import numpy as np: This imports the NumPy library, which is used for mathematical operations, especially those involving arrays (like grids or matrices).
import matplotlib.pyplot as plt: This imports the Matplotlib library, which is used for creating plots (graphs). We will use it to visualize the data.

In [ ]:
import numpy as numpy
import matplotlib.pyplot as plt

## Fractal Dimension Function
def *fractal_dimension_3d(grid)*:: This defines a function named *fractal_dimension_3d* that takes one input, grid, which is a 3D array representing the fractal object. This grid will be a 3D numpy array where:
 - 1 represents occupied space (the fractal part),
 - 0 represents empty space.

In [ ]:
def fractal_dimension_3d(grid):
    """
    Calculate the fractal dimension of a 3D object using the box-counting method.
    Args:
        grid: A 3D numpy array where 1 indicates the object and 0 is empty space.
    Returns:
        fractal_dimension: The fractal dimension of the object.
    """

## Ensuring the Grid is Binary
*grid = (grid > 0)*: This line ensures that the grid contains only 1s (for occupied) and 0s (for empty). Any positive value in the grid gets turned into 1, and 0 remains as 0. This makes sure we only have two values: 0 (empty) and 1 (filled).

In [ ]:
grid = (grid > 0)

## Grid Size and Validation
*size = grid.shape[0]*: This gets the size of the grid. Since the grid is assumed to be a cube (NxNxN), we only need the size of one dimension (all three dimensions are the same).
*assert grid.shape[0] == grid.shape[1] == grid.shape[2]*: This line checks if the grid is indeed a cube. If it's not, it will stop the program and show the error message "Grid must be a cube!".

In [ ]:
size = grid.shape[0]  # Assume the grid is a cube (NxNxN)
assert grid.shape[0] == grid.shape[1] == grid.shape[2], "Grid must be a cube!"

## Define Scales (Box Sizes)
*np.logspace(0, np.log2(size), num=10, base=2, endpoint=False)*: This creates an array of 10 scales using a logarithmic scale, spaced between 1 and the size of the grid (in powers of 2).
 - np.log2(size) gives the logarithmic scale based on the grid size.
 - base=2 makes the scale exponential in base 2 (like powers of 2).
 - num=10 creates 10 values.
 - endpoint=False ensures the last value isn't included in the range.

*np.floor(...).astype(int)*: This rounds the scale values down and converts them into integers.

*scales = scales[scales > 1]*: This removes any scale that is less than or equal to 1, because we want meaningful box sizes.

*scales = [scale for scale in scales if size % scale == 0]*: This keeps only the scales that divide the grid size exactly (without leaving a remainder). If a scale doesn't fit evenly into the grid, it is removed.

In [ ]:
scales = np.floor(np.logspace(0, np.log2(size), num=10, base=2, endpoint=False)).astype(int)
scales = scales[scales > 1]  # Exclude box size 1 (too small)
scales = [scale for scale in scales if size % scale == 0]  # Keep only divisible scales


## Counting Non-Empty Boxes at Each Scale
- Ns = []: This initializes an empty list Ns where we will store the number of non-empty boxes at each scale.

- *for scale in scales*:: This starts a loop to process each scale.

Inside the loop:

- *grid.reshape(...)*: This reshapes the grid into smaller blocks of the current scale. We are dividing the grid into smaller cubes of size (scale x scale x scale). For example, if the scale is 3, we divide the grid into smaller cubes of size 3x3x3.
- *.max(axis=(1, 3, 5))*: This reduces each block to its maximum value along each dimension. If there is any 1 in the block, it becomes 1, otherwise, it stays 0. This ensures that if there is any part of the object in a box, the box is considered "occupied."
- *np.sum(sub_boxes > 0)*: This counts how many of these blocks are "occupied" (i.e., contain a 1). This count is appended to the Ns list.

In [ ]:
Ns = []  # Store the number of boxes at each scale

for scale in scales:
        sub_boxes = (
            grid.reshape(size // scale, scale,
                         size // scale, scale,
                         size // scale, scale)
                .max(axis=(1, 3, 5))
        )
        Ns.append(np.sum(sub_boxes > 0))

## Checking for Enough Scales
This checks if there are enough valid scales (at least 2) for a reliable fit. If there are fewer than 2, it raises an error and tells you to adjust the grid size or scales.

In [ ]:
if len(scales) < 2:
    raise ValueError("Not enough valid scales for a reliable fit. Increase the grid size or refine scales.")


## Calculating the Fractal Dimension
*np.log(scales)*: This takes the natural logarithm of the scales (logarithmic transformation of the scale values).
*np.log(Ns)*: This takes the natural logarithm of the counts of occupied boxes (logarithmic transformation of the Ns values).
*np.polyfit(log_scales, log_Ns, 1)*: This fits a straight line to the data in a log-log plot (using linear regression). The slope of this line corresponds to the fractal dimension. The 1 means we're fitting a straight line (1st degree polynomial).

In [ ]:
log_scales = np.log(scales)
log_Ns = np.log(Ns)
coeffs = np.polyfit(log_scales, log_Ns, 1)

## Extracting the Fractal Dimension and Plotting
The slope (fractal dimension) is -coeffs[0] because the slope of a log-log plot of the form 
𝑁
(
𝑠
)
∼
𝑠
−
𝐷
N(s)∼s 
−D
  is negative.

In [ ]:
fractal_dimension = -coeffs[0]

## Plotting the Results
- *plt.figure(figsize=(6, 4))*: This creates a new plot with a size of 6x4 inches.
- *plt.plot(log_scales, log_Ns, 'o', mfc='none', label='Data')*: This plots the log-log data points (log_scales vs. log_Ns) as circles.
- *plt.plot(...)*: This plots the fitted line from the regression with the fractal dimension D shown in the legend.
- *plt.xlabel(), plt.ylabel()*: These set the labels for the x and y axes.
- *plt.legend()*: This adds a legend to the plot.
- *plt.title()*: This adds a title to the plot.
- *plt.grid()*: This adds a grid to the plot.
- *plt.show()*: This displays the plot.

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(log_scales, log_Ns, 'o', mfc='none', label='Data')
plt.plot(log_scales, np.polyval(coeffs, log_scales), label=f'Fit (D = {fractal_dimension:.3f})')
plt.xlabel('log(Scale)')
plt.ylabel('log(Number of Boxes)')
plt.legend()
plt.title('3D Fractal Dimension')
plt.grid()
plt.show()

## Return Fractal Dimension
Finally, the function returns the calculated fractal dimension.

In [ ]:
return fractal_dimension


## Sierpinski Carpet Function
- This function creates a 3D Sierpinski carpet fractal by repeatedly removing cubes from the center.
- *np.ones((size, size, size), dtype=bool)* creates an initial 3D grid filled with 1s (occupied).
- The loop removes parts of the grid by setting portions of it to 0 (empty space), following the Sierpinski fractal pattern.

In [ ]:
def sierpinski_carpet_3d(n, size):
    """Create a 3D Sierpinski carpet fractal."""
    grid = np.ones((size, size, size), dtype=bool)
    for _ in range(n):
        step = size // 3
        grid[step:2*step, step:2*step, :] = 0
        grid[step:2*step, :, step:2*step] = 0
        grid[:, step:2*step, step:2*step] = 0
        size //= 3
    return grid

## Example Usage
- *size = 243*: Defines the size of the grid.
- *grid = sierpinski_carpet_3d(5, size)*: Generates a 3D Sierpinski fractal with 5 iterations.
- *fractal_dimension_3d(grid)*: Computes the fractal dimension of the generated grid.
- *print(f"Fractal Dimension: {fractal_dim:.3f}")*: Prints the computed fractal dimension.


In [ ]:
size = 243  # Larger size allows more valid scales
grid = sierpinski_carpet_3d(5, size)  # Increase iterations for finer detail

try:
    fractal_dim = fractal_dimension_3d(grid)
    print(f"Fractal Dimension: {fractal_dim:.3f}")
except ValueError as e:
    print("Error:", e)